# Amazon S3 Fundamentals With Python Boto3 For Complete Beginners
## Learn how to use cloud resources in your Python scripts
![](images/pexels.jpg)

### Setup

In [4]:
import logging
import time
import warnings

import catboost as cb
import datatable as dt
import joblib
import lightgbm as lgbm
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import seaborn as sns
import shap
import umap
import umap.plot
import xgboost as xgb
from optuna.samplers import TPESampler
from sklearn.compose import *
from sklearn.impute import *
from sklearn.metrics import *
from sklearn.model_selection import *
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import *

logging.basicConfig(
    format="%(asctime)s - %(message)s", datefmt="%d-%b-%y %H:%M:%S", level=logging.INFO
)
optuna.logging.set_verbosity(optuna.logging.WARNING)
warnings.filterwarnings("ignore")
pd.set_option("float_format", "{:.5f}".format)

### Introduction

### Setup an account

Right, let's start with creating your AWS account, if you haven't already. Nothing unusual, just follow the steps from this [link](https://aws.amazon.com/free/?trk=ps_a131L0000085EJvQAM&trkCampaign=acq_paid_search_brand&sc_channel=ps&sc_campaign=acquisition_US&sc_publisher=google&sc_category=core-main&sc_country=US&sc_geo=NAMER&sc_outcome=acq&sc_detail=aws&sc_content=Brand_Core_aws_e&sc_segment=432339156150&sc_medium=ACQ-P|PS-GO|Brand|Desktop|SU|Core-Main|Core|US|EN|Text&s_kwcid=AL!4422!3!432339156150!e!!g!!aws&ef_id=EAIaIQobChMIxa2ogpvA9QIVxP7jBx2iAwgNEAAYASAAEgJGR_D_BwE:G:s&s_kwcid=AL!4422!3!432339156150!e!!g!!aws&all-free-tier.sort-by=item.additionalFields.SortRank&all-free-tier.sort-order=asc&awsf.Free%20Tier%20Types=*all&awsf.Free%20Tier%20Categories=*all):

![](images/sign_up.gif)

Then, we will go to the [AWS IAM (Identity and Access Management) console](https://console.aws.amazon.com/), which is the place we will be doing most of the work.

![](images/aws_console.gif)

From the console, you can easily switch between different AWS servers, create users, add policies and allow access to your user account. We will do each one by one.

### Create a user

![](images/create_user.gif)

![](images/credentials.png)

### Create a bucket

![](images/create_bucket.gif)

### Create a policy and add it to your user

![](images/create_policy.gif)

![](images/add_permission.gif)

### Download AWS CLI and configure your user

![](images/download_cli.gif)

![](images/configure_aws.gif)

### Upload your files and copy URI

![](images/upload_file.gif)

![](images/copy_uri.gif)

### Using Python Boto3 to download files from S3 bucket

### Other operations with Boto3